In [1]:
def eval_atom_cnt(task, pred):
    # task = atom count
    return pred==task['output']

def eval_dimensions(task, pred):
    # task = dimensions, pred is a list
    if "lengths of the lattice vectors" in task['input']:
        lengths = [float(x) for x in task['output'].split(',')]
        mse = np.mean((lengths - pred) ** 2)
        return mses
    else:
        angles = [float(x) for x in task['output'].split(',')]
        mae = np.mean(abs(angles - pred))
        return mae

def eval_atom_name(task, pred):
    # task = atom name
    return pred==task['output']

def eval_spacegroup(task, pred):
    # task = space group
    return pred==task['output']

def eval_cell_volume(task, pred):
    # task = cell_volume
    return abs(task['output']-pred)

def eval_formula(task, pred):
    # task = formula
    return task['output']==pred

def eval_replace(task, pred):
    # task = replace
    return task['output']==pred

In [2]:
def verify_gen_format(output):
    lines = output.split('\n')
    if len(lines)%2:
        return 0
    
    lengths = lines[0]
    angles = lines[1]
    for i in range(2, len(lines), 2):
        

IndentationError: expected an indented block (389705309.py, line 9)

In [ ]:
def eval_infill():

def eval_dimensions_gen():


In [ ]:
def get_structure(cif_str):
    structure = Structure.from_str(cif_str, fmt="cif")

#     structure.translate_sites(
#         indices=range(len(structure.sites)), vector=np.random.uniform(size=(3,))
#     )
    return structure

def get_crystal_string_nate(structure):
    lengths = structure.lattice.parameters[:3]
    angles = structure.lattice.parameters[3:]
    atom_ids = structure.species
    frac_coords = structure.frac_coords

    crystal_str = \
        " ".join(["{0:.1f}".format(x) for x in lengths]) + "\n" + \
        " ".join([str(int(x)) for x in angles]) + "\n" + \
        "\n".join([
            str(t) + "\n" + " ".join([
                "{0:.2f}".format(x) for x in c
            ]) for t,c in zip(atom_ids, frac_coords)
        ])

    return crystal_str

def get_crystal_string(structure):
    lengths = structure.lattice.parameters[:3]
    angles = structure.lattice.parameters[3:]
    atom_ids = structure.species
    frac_coords = structure.frac_coords

    lens = [float("{0:.1f}".format(x)) for x in lengths]
    angs = [int(x) for x in angles]
    coords = "\n".join([
            str(t) + "\n" + " ".join([
                "{0:.2f}".format(x) for x in c
            ]) for t,c in zip(atom_ids, frac_coords)
    ])

    crystal = {
        "lengths": lens,
        "angles": angs,
        "coordinates": coords
    }

    return json.dumps(crystal)

In [1]:
import json
import numpy as np
import pandas as pd
import vllm
import torch
import string
import re
import collections
from sklearn.metrics import f1_score
import Levenshtein
import os
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, LlamaTokenizer
import datetime
from functools import reduce
from huggingface_hub import login

# daddy_df = []

In [2]:
valfile = []
with open("/scratch/cse/btech/cs1200448/MatLlama/ift_cif/val.jsonl", 'r') as f:
    valfile = [json.loads(line) for line in f.readlines()]

valinputs = [f"<|im_start|>system\n{i['system']}<|im_end|>\n"+f"<|im_start|>question\n{i['input']}<|im_end|>\n"+"<|im_start|>answer\n" for i in valfile]

init_seed = 2
seed = 1
num_seeds = 3

kwargs = {
    "model": "/scratch/cse/btech/cs1200448/MatLlama/ft-checkpoints/ift_cif_hf",
    "tokenizer": "/scratch/cse/btech/cs1200448/llama-weights/7b/tokenizer.model",
    "trust_remote_code": True,
    "tensor_parallel_size": 1,
    "seed":seed,
    # "gpu_memory_utilization":args.mem_util,
}

client = vllm.LLM(**kwargs)

OSError: /scratch/cse/btech/cs1200448/MatLlama/ft-checkpoints/ift_cif5/1650/iter_0004000 does not appear to have a file named config.json. Checkout 'https://huggingface.co//scratch/cse/btech/cs1200448/MatLlama/ft-checkpoints/ift_cif5/1650/iter_0004000/tree/None' for available files.

In [ ]:
# for _ in range(1, num_seeds+1):
seed = seed * init_seed

response = client.generate(valinputs, sampling_params=vllm.SamplingParams(
        skip_special_tokens=True,
        best_of=1,
        presence_penalty=0.0,
        frequency_penalty=1.0,
        top_k=50,
        top_p=1.0,
        temperature=0.75,
        stop=["<|im_start|>", "<|im_end|>"],
        use_beam_search=False,
        max_tokens=300,
        logprobs=2
    ))
predictions = [i.outputs[0].text for i in response]

def most_similar_answer(a,answer_set):
    a = a.strip().replace(' ', '')
    if(a in answer_set):
        return a
    dis = [Levenshtein.distance(a,x) for x in answer_set]
    idx = np.argmin(dis)
    return answer_set[idx]

for _, sample in enumerate(valfile):
    task = sample['task']
    system = sample['system']
    output = predictions[_]
    print(sample['input'])
    print(sample['output'])
    print("--------")
    print(output)
    break

In [ ]:
   # if len(output) == 0: # cases with prompt length > ctxlen
        #     continue
        # if 'SOFC' in system:
        #     task = 'sc'
        #     dataset = 'sofc_sent'
        # elif 'named entity' in system:
        #     task = 'ner'
        #     if 'b-dsc' in system:
        #         dataset = 'matscholar'
        #     elif 'b-device' in system:
        #         dataset = 'sofc_token'
        #     elif 'doping' in system:
        #         dataset = 'sc_comics'
        # elif 'synthesis action' in system:
        #     task = 'sar'
        #     dataset = 'synthesis_actions'
        # elif 'slots' in system:
        #     task = 'sf'
        #     dataset = 'sofc_token'
        # elif 'event, identify the roles of the arguments' in system:
        #     task = 'ee'
        #     dataset = 'sc_comics'
        # elif 'extract relation' in system:
        #     task = 're'
        #     for ds in kw[task]:
        #         if kw[task][ds][0] in system:
        #             dataset = ds
        #             break   
        # elif 'inorganic glass' in system:
        #     task = 'pc'
        #     dataset = 'glass_non_glass'
        # else:
        #     task = 'qna'
        #     dataset = 'squad'
        
        # assert task != None
        # assert dataset != None
        
        # if task in ["ner", "sf", "ee", "sar"]:
        #     answer = [i.split(" : ") for i in answer.lower().split('\n')]
        #     try:
        #         temp = [[j.strip().lower() for j in i.split(":")] for i in output.split('\n')]
        #         temp = [i for i in temp if len(i) == 2]
        #         output = temp
        #     except:
        #         print(f"for tc {_}, output pattern mismatched. {output}")
        # else:
        #     answer = answer.strip().lower()
        #     output = output.strip().lower()
    
        # out_dict[task][dataset].append((answer, output))

    # scores = {i : [] for i in tasks}
    # scores["ner"] = {i : (0,0) for i in ["matscholar", "sofc_token", "sc_comics"]}
    # scores["pc"] = {i : (0,0) for i in ["glass_non_glass"]}
    # scores["sf"] = {i : (0,0) for i in ["sofc_token"]}
    # scores["ee"] = {i : (0,0) for i in ["sc_comics"]}
    # scores["re"] = {i : (0,0) for i in ["structured_re", "sc_comics"]}
    # scores["sar"] = {i : (0,0) for i in ["synthesis_actions"]}
    # scores["sc"] = {i : (0,0) for i in ["sofc_sent"]}
    # scores["qna"] = {i : (0,0) for i in ["squad"]}

    # def evaluate(task, dataset):
    #     all_gt = []
    #     all_pred = []
    #     for gt, pred in out_dict[task][dataset]:
    #         dict = {}
    #         for word, entity in gt:
    #             dict[word] = [entity, 'O']
                    
    #         for word, entity in pred:
    #             to_put = most_similar_answer(entity, kw[task][dataset])
    #             if word in dict:
    #                 dict[word][1] = to_put
    #             else:
    #                 dict[word] = ['O', to_put]
                    
    #         for i in dict.values():
    #             all_gt.append(i[0])
    #             all_pred.append(i[1])
    
    #     micro_f1 = f1_score(all_gt, all_pred, average='micro', labels = list(kw[task][dataset]))
    #     macro_f1 = f1_score(all_gt, all_pred, average='macro', labels = list(kw[task][dataset]))
    #     scores[task][dataset] = micro_f1, macro_f1
    #     return micro_f1, macro_f1

    # def evaluate_pc(dataset):
    #     out_dict['pc'][dataset] = [(most_similar_answer(i[0], kw['pc'][dataset]), most_similar_answer(i[1], kw['pc'][dataset])) for i in out_dict['pc'][dataset]]
    
    #     micro_f1 = f1_score(*list(zip(*out_dict['pc'][dataset])), average='micro', labels = list(kw['pc'][dataset]))
    #     macro_f1 = f1_score(*list(zip(*out_dict['pc'][dataset])), average='macro', labels = list(kw['pc'][dataset]))
    #     scores['pc'][dataset] = micro_f1, macro_f1
    #     return micro_f1, macro_f1
    
    # def evaluate_sc(dataset):
    #     out_dict['sc'][dataset] = [(most_similar_answer(i[0], kw['sc'][dataset]), most_similar_answer(i[1], kw['sc'][dataset])) for i in out_dict['sc'][dataset]]
        
    #     micro_f1 = f1_score(*list(zip(*out_dict['sc'][dataset])), average='micro', labels = list(kw['sc'][dataset]))
    #     macro_f1 = f1_score(*list(zip(*out_dict['sc'][dataset])), average='macro', labels = list(kw['sc'][dataset]))
    #     scores['sc'][dataset] = micro_f1, macro_f1
    #     return micro_f1, macro_f1
    
    # def evaluate_re(dataset):
    #     out_dict['re'][dataset] = [(most_similar_answer(i[0], kw['re'][dataset]), most_similar_answer(i[1], kw['re'][dataset])) for i in out_dict['re'][dataset]]
        
    #     micro_f1 = f1_score(*list(zip(*out_dict['re'][dataset])), average='micro', labels = list(kw['re'][dataset]))
    #     macro_f1 = f1_score(*list(zip(*out_dict['re'][dataset])), average='macro', labels = list(kw['re'][dataset]))
    #     scores['re'][dataset] = micro_f1, macro_f1
    #     return micro_f1, macro_f1

    # def normalize_answer(s):
    #     """Lower text and remove punctuation, articles and extra whitespace."""
    #     def remove_articles(text):
    #         regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    #         return re.sub(regex, ' ', text)
    #     def white_space_fix(text):
    #         return ' '.join(text.split())
    #     def remove_punc(text):
    #         exclude = set(string.punctuation)
    #         return ''.join(ch for ch in text if ch not in exclude)
    #     def lower(text):
    #         return text.lower()
    #     return white_space_fix(remove_articles(remove_punc(lower(s))))
        
    # def get_tokens(s):
    #     if not s: return []
    #     return normalize_answer(s).split()
    
    
    # def compute_exact(a_gold, a_pred):
    #     return int(normalize_answer(a_gold) == normalize_answer(a_pred))
    
    # def compute_f1(a_gold, a_pred):
    #     gold_toks = get_tokens(a_gold)
    #     pred_toks = get_tokens(a_pred)
    #     common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    #     num_same = sum(common.values())
    #     if len(gold_toks) == 0 or len(pred_toks) == 0:
    #     # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    #         return int(gold_toks == pred_toks)
    #     if num_same == 0:
    #         return 0
    #     precision = 1.0 * num_same / len(pred_toks)
    #     recall = 1.0 * num_same / len(gold_toks)
    #     f1 = (2 * precision * recall) / (precision + recall)
    #     return f1
    
    # def get_raw_scores(dataset):
    #     exact_scores = []
    #     f1_scores = []
    #     for article in dataset:
    #         gold_answers = [article[0] if normalize_answer(article[0]) else None]
    #         if not gold_answers[0]:
    #             print("sad")
    #             gold_answers = ['']
    #         a_pred = article[1]
    #         exact_scores.append(max(compute_exact(a, a_pred) for a in gold_answers))
    #         f1_scores.append(max(compute_f1(a, a_pred) for a in gold_answers))
    #     return exact_scores, f1_scores
    
    # def make_eval_dict(exact_scores, f1_scores):
    #     total = len(exact_scores)
    #     assert len(exact_scores) == len(f1_scores)
    #     return collections.OrderedDict([
    #         ('exact', 1.0 * sum(k for k in exact_scores) / total),
    #         ('f1', 1.0 * sum(k for k in f1_scores) / total),
    #         ('total', total),
    #     ])

    
    # def most_similar_answer(a,answer_set):
    #     a = a.strip().replace(' ', '')
    #     if(a in answer_set):
    #         return a
    #     dis = [Levenshtein.distance(a,x) for x in answer_set]
    #     idx = np.argmin(dis)
    #     return answer_set[idx]
    
    # def evaluate_qna(dataset):
    #     exact_raw, f1_raw = get_raw_scores(out_dict['qna'][dataset])
    #     out_eval = make_eval_dict(exact_raw, f1_raw)
    #     scores['qna'][dataset] = out_eval['f1'], out_eval['exact']
        
    #     return out_eval

    # if len(out_dict['qna']['squad']) > 0:
    #     evaluate_qna('squad')
    # if len(out_dict['ner']['matscholar']) > 0:
    #     evaluate('ner', 'matscholar')
    # if len(out_dict['ner']['sofc_token']) > 0:
    #     evaluate('ner', 'sofc_token')
    # if len(out_dict['ner']['sc_comics']) > 0:
    #     evaluate('ner', 'sc_comics')
    # if len(out_dict['sar']['synthesis_actions']) > 0:
    #     evaluate('sar', 'synthesis_actions')
    # if len(out_dict['ee']['sc_comics']) > 0:
    #     evaluate('ee', 'sc_comics')
    # if len(out_dict['sf']['sofc_token']) > 0:
    #     evaluate('sf', 'sofc_token')
    # if len(out_dict['sc']['sofc_sent']) > 0:
    #     evaluate_sc('sofc_sent')
    # if len(out_dict['pc']['glass_non_glass']) > 0:
    #     evaluate_pc('glass_non_glass')
    # if len(out_dict['re']['structured_re']) > 0:
    #     evaluate_re('structured_re')
    # if len(out_dict['re']['sc_comics']) > 0:
    #     evaluate_re('sc_comics')

    # df = pd.DataFrame.from_dict({(i,j): scores[i][j] for i in scores.keys() for j in scores[i].keys()},orient='index', columns = ['micro-f1/f1', 'macro-f1/em'])
    # df = df.apply(lambda x: 100 * round(x, 5))

#     # scores.pop(('ner', 'sc_comics'))
#     # df = pd.DataFrame(scores).apply(lambda x : 100 * round(x,5))
#     # df = df.fillna('-')
#     print(df.to_string())
#     daddy_df.append(df)

# df_sum = reduce(lambda x, y: x.add(y, fill_value=0), daddy_df)
# df_mean = df_sum / len(daddy_df)
# df_concat = pd.concat(daddy_df)
# df_std = df_concat.groupby(df_concat.index).std()

# df_mean.columns = pd.MultiIndex.from_tuples([(i, f"mean") for i in df_mean.columns])
# df_std.columns = pd.MultiIndex.from_tuples([(i, f"std") for i in df_std.columns])

# combined_df = pd.concat([df_mean, df_std], axis=1)
# combined_df = combined_df.sort_index(axis=1).apply(lambda x : round(x,2))


# print(f"===eval over average of {args.num_seeds} runs===")
# print(combined_df.to_string())
# prefix = args.checkpoint.split('/')[-3] + '-' + args.checkpoint.split('/')[-2]

# now = datetime.datetime.now()
# datetimestring = now.strftime("%y-%m-%d-%H-%M")

# combined_df.to_csv(f'/home/cse/btech/cs1200448/MatLlama/scripts/csvs/{prefix}-{datetimestring}.csv')

# df_sum = reduce(lambda x, y: x.add(y, fill_value=0), daddy_df)
# df_mean = df_sum / len(daddy_df)
# df_concat = pd.concat(daddy_df)
# df_std = df_concat.groupby(df_concat.index).std()

# df_mean.columns = pd.MultiIndex.from_tuples([(i[0], f"{i[1]}_mean") for i in df_mean.columns])
# df_std.columns = pd.MultiIndex.from_tuples([(i[0], f"{i[1]}_std") for i in df_std.columns])

# combined_df = pd.concat([df_mean, df_std], axis=1)
# combined_df = combined_df.sort_index(axis=1).apply(lambda x : round(x,5))


# print(f"===eval over average of {args.num_seeds} runs===")
# print(combined_df.to_string())

# prefix = args.checkpoint.split('/')[-1]

# now = datetime.datetime.now()
# datetimestring = now.strftime("%y-%m-%d-%H-%M")

# combined_df.to_csv(f'csvs/{prefix}-{datetimestring}.csv')